<a href="https://colab.research.google.com/github/brenoakihiromorimoto/portf-lio/blob/main/%237DaysOfCode/dia_1_Data_Cleaning_and_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# #7DaysOfCode - Ciência de Dados 1/7: Data Cleaning and Preparation
Antes de começar, me responda uma pergunta: com o que você acha que Data Scientists gastam mais tempo durante seu trabalho?

Eu queria poder mentir pra você e dizer que passamos a maior parte do nosso tempo criando modelos altamente complexos e brincando com o que há de mais avançado tecnologicamente.

Eu queria, ainda, poder dizer que as bases de dados que você irá trabalhar já estão limpas, bem processadas e prontas para que você possa criar visualizações e análises poderosas.

Mas, na verdade, não é bem isso. O que passamos a maior parte do tempo fazendo é a preparação do dado, mais especificamente nas partes de limpeza e transformação.

Em uma pesquisa com cerca de 80 cientistas de dados, as etapas de preparação de dados foram responsáveis por consumir mais de 80% do tempo deles, reforçando a necessidade de desenvolver a habilidade de tratar dados e deixá-los prontos para a parte mais divertida: criar modelos e análises.

Por isso, quero te dar um ótimo desafio para desenvolver e aprimorar essa skill tão importante para toda pessoa cientista de dados.

Neste primeiro dia, eu te proponho baixar um dataset do portal do CEAPS (Cota para Exercício da Atividade Parlamentar dos Senadores) e aplicar processos de tratamento e limpeza de dados nele (processo conhecido como Data Wrangling).

Basicamente, o CEAPS contém todos os gastos que senadores brasileiros declararam, divididos por ano.

Esse tipo de dado é tão importante que já criou várias iniciativas interessantes, como a Operação Serenata de Amor, que aplica Inteligência Artificial para analisar gastos de deputados brasileiros, e que já foi capaz de identificar vários usos indevidos do dinheiro público.

Imagina as coisas que você pode criar!

Os dados do CEAPS contêm uma série de problemas que podem dificultar a criação de análises mais aprofundadas.

Uma das primeiras coisas que você pode fazer é identificar tais inconsistências, como campos que possuem valores nulos ou duplicados, converter campos de data que estão sendo carregados como texto, corrigir valores monetários, nomes incorretos, formatar campos de CNPJ, etc.
DICA
Os dados do CEAPS estão divididos por ano.

Que tal juntar dados de vários anos em um grande dataset e aplicar técnicas de limpeza e processamento dos dados?

Você poderia pegar dados dos últimos quatro anos e aplicar o que usou nesse exercício.

Além disso, também proponho que você documente o seu processo de tratamento dos dados (pode ser no próprio Jupyter Notebook). Assim, qualquer pessoa que consumir o seu trabalho saberá qual foi sua intuição e as técnicas utilizadas, além de facilitar a reprodutibilidade.

Tire o maior proveito possível dos comentários no código!

Não existe uma receita de bolo para quais técnicas utilizar na limpeza de dados, isso varia de projeto para projeto. Contudo, existem algumas coisas que você pode fazer inicialmente.

Lidar com dados nulos (deletar ou imputar um valor novo, por exemplo); remover colunas que não trazem nenhuma informação; processar datas que estão em formato incorreto; alterar o tipo da coluna (uma coluna que é numérica está como texto no Pandas); remover duplicados; dentre outras.

Esse post da Tableau traz algumas dicas sobre técnicas que você pode utilizar.

# Importar Bibliotecas

In [2]:
from google.colab import drive
drive.mount('/gdrive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

Mounted at /gdrive


# Datasets de despesas de cota parlamentar dos senadores.
* Os datasets analisados foram entre 2019-2022.
* Detectar o tipo de encoding dos datasets.

## Detectar o tipo de encoding dos datasets.

In [3]:
import chardet

link = "/gdrive/MyDrive/Colab Notebooks/#7DaysOfCode/"

nome_datasets = ["despesa_ceaps_2019.csv",
                 "despesa_ceaps_2020.csv",
                 "despesa_ceaps_2021.csv",
                 "despesa_ceaps_2022.csv"]
for dataset in nome_datasets:
  with open(link+dataset, 'rb') as rawdata:
      result = chardet.detect(rawdata.read(1000000))
      print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


* O encoding do despesa_ceaps_2019 até despesa_ceaps_2021 é 'ISO-8859-1'.
* O encoding do despesa_ceaps_2022 é Windows-1252.

## Importar datasets

In [4]:
despesa_ceaps_2019 = pd.read_csv(link+"despesa_ceaps_2019.csv", 
                                 encoding = 'ISO-8859-1',
                                 error_bad_lines=True, sep=';', )

despesa_ceaps_2020 = pd.read_csv(link+"despesa_ceaps_2020.csv", 
                                 encoding = 'ISO-8859-1',
                                 error_bad_lines=True, sep=';')

despesa_ceaps_2021 = pd.read_csv(link+"despesa_ceaps_2021.csv", 
                                 encoding = 'ISO-8859-1',
                                 error_bad_lines=True, sep=';')

despesa_ceaps_2022 = pd.read_csv(link+"despesa_ceaps_2022.csv", 
                                 encoding = 'Windows-1252',
                                 error_bad_lines=True, sep=';')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
despesa_ceaps_2019.head(3)

ULTIMA ATUALIZACAO  \
ANO  MES SENADOR      TIPO_DESPESA                                       CNPJ_CPF           FORNECEDOR DOCUMENTO DATA       DETALHAMENTO                                        VALOR_REEMBOLSADO   
2019 1   ACIR GURGACZ Aluguel de imóveis para escritório político, co... 05.914.650/0001-66 ENERGISA   006582758 04/01/2019 Despesa com pagamento de energia elétrica do es...              66,02   
                                                                                                       006582755 04/01/2019 Despesa com pagamento de energia elétrica do es...             139,98   

                                                                                                                                                                               06/08/2021 02:08  
ANO  MES SENADOR      TIPO_DESPESA                                       CNPJ_CPF           FORNECEDOR DOCUMENTO DATA       DETALHAMENTO                                          COD_DOCUMENTO  
2019 1   ACIR GURGACZ Aluguel de imóveis para escritório político, co... 05.914.650/0001-66 ENERGISA   006582758 04/01/2019 Despesa com pagamento de energia elétrica do es...          2116543  
                                                                                                       006582755 04/01/2019 Despesa com pagamento de energia elétrica do es...          2116546

In [6]:
despesa_ceaps_2020.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 14091 entries, ('ANO', 'MES', 'SENADOR', 'TIPO_DESPESA', 'CNPJ_CPF', 'FORNECEDOR', 'DOCUMENTO', 'DATA', 'DETALHAMENTO') to ('2020', '12', 'ZEQUINHA MARINHO', 'Passagens aéreas, aquáticas e terrestres nacionais', '22.052.777/0001-32', 'Exceller Tour', 'BQWCVK', '18/12/2020', 'Companhia Aérea: GOL, Localizador: BQWCVK. Passageiros: JOSÉ DA CRUZ MARINHO (Matrícula 341141, PARLAMENTAR), Voo: 1822 - BSB/MARABÁ - 19/12/2020;')
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ULTIMA ATUALIZACAO  14091 non-null  object
 1   14/04/2022 02:00    14091 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [7]:
despesa_ceaps_2020.head(3)

ULTIMA ATUALIZACAO  \
ANO  MES SENADOR      TIPO_DESPESA                                       CNPJ_CPF           FORNECEDOR DOCUMENTO DATA       DETALHAMENTO                                        VALOR_REEMBOLSADO   
2020 1   ACIR GURGACZ Aluguel de imóveis para escritório político, co... 05.914.650/0001-66 ENERGISA   14764742  06/01/2020 Despesa com energia elétrica para uso do escrit...             122,62   
                                                                                                       14764741  06/01/2020 Despesa com energia elétrica para uso do escrit...             137,75   

                                                                                                                                                                               14/04/2022 02:00  
ANO  MES SENADOR      TIPO_DESPESA                                       CNPJ_CPF           FORNECEDOR DOCUMENTO DATA       DETALHAMENTO                                          COD_DOCUMENTO  
2020 1   ACIR GURGACZ Aluguel de imóveis para escritório político, co... 05.914.650/0001-66 ENERGISA   14764742  06/01/2020 Despesa com energia elétrica para uso do escrit...          2141820  
                                                                                                       14764741  06/01/2020 Despesa com energia elétrica para uso do escrit...          2141818

In [8]:
despesa_ceaps_2021.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 16809 entries, ('ANO', 'MES', 'SENADOR', 'TIPO_DESPESA', 'CNPJ_CPF', 'FORNECEDOR', 'DOCUMENTO', 'DATA', 'DETALHAMENTO') to ('2021', '12', 'ZEQUINHA MARINHO', 'Passagens aéreas, aquáticas e terrestres nacionais', '22.052.777/0001-32', 'Exceller Tour', 'LM6FSK', '29/12/2021', 'Companhia Aérea: AZUL, Localizador: LM6FSK. Passageiros: JOSÉ DA CRUZ MARINHO (Matrícula 341141, PARLAMENTAR), Voo: 4460 - BELÉM/MARABÁ - 30/12/2021;')
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ULTIMA ATUALIZACAO  16809 non-null  object
 1   14/04/2022 02:00    16809 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [9]:
despesa_ceaps_2021.head(3)

ULTIMA ATUALIZACAO  \
ANO  MES SENADOR      TIPO_DESPESA                                       CNPJ_CPF           FORNECEDOR                                         DOCUMENTO   DATA       DETALHAMENTO                                        VALOR_REEMBOLSADO   
2021 1   ACIR GURGACZ Aluguel de imóveis para escritório político, co... 05.914.650/0001-66 ENERGISA                                           023.489.627 21/01/2021 Pagamento de energia elétrica para uso do escri...              75,29   
                                                                         062.135.728-64     FERNANDO WALDEIR PACINI e ANA LUCIA DA SILVA SI... 01/2021     05/01/2021 Despesa com pagamento de aluguel de imóvel para...               1000   

                                                                                                                                                                                                                         14/04/2022 02:00  
ANO  MES SENADOR      TIPO_DESPESA                                       CNPJ_CPF           FORNECEDOR                                         DOCUMENTO   DATA       DETALHAMENTO                                          COD_DOCUMENTO  
2021 1   ACIR GURGACZ Aluguel de imóveis para escritório político, co... 05.914.650/0001-66 ENERGISA                                           023.489.627 21/01/2021 Pagamento de energia elétrica para uso do escri...          2158003  
                                                                         062.135.728-64     FERNANDO WALDEIR PACINI e ANA LUCIA DA SILVA SI... 01/2021     05/01/2021 Despesa com pagamento de aluguel de imóvel para...          2157367

In [10]:
despesa_ceaps_2022.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3750 entries, ('ANO', 'MES', 'SENADOR', 'TIPO_DESPESA', 'CNPJ_CPF', 'FORNECEDOR', 'DOCUMENTO', 'DATA', 'DETALHAMENTO') to ('2022', '4', 'ZEQUINHA MARINHO', 'Passagens aéreas, aquáticas e terrestres nacionais', '22.052.777/0001-32', 'Exceller Tour', 'FRFFFI', '06/04/2022', 'Companhia Aérea: LATAM, Localizador: FRFFFI. Passageiros: JOSÉ DA CRUZ MARINHO (Matrícula 341141, PARLAMENTAR), Voo: 3705 - BELÉM/BSB - 17/04/2022;')
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ULTIMA ATUALIZACAO  3750 non-null   object
 1   14/04/2022 02:01    3750 non-null   object
dtypes: object(2)
memory usage: 401.9+ KB


In [11]:
despesa_ceaps_2022.head(3)

ULTIMA ATUALIZACAO  \
ANO  MES SENADOR      TIPO_DESPESA                                       CNPJ_CPF           FORNECEDOR                    DOCUMENTO         DATA       DETALHAMENTO                                        VALOR_REEMBOLSADO   
2022 1   ACIR GURGACZ Aluguel de imóveis para escritório político, co... 004.948.028-63     GILBERTO PISELO DO NASCIMENTO 001/22            03/01/2022 Despesa com pagamento de aluguel de imóvel para...               6000   
                      Divulgação da atividade parlamentar                26.320.603/0001-64 INFORMANAHORA                 000000000000310/A 04/01/2022 Despesa com divulgação da atividade parlamentar...               1500   

                                                                                                                                                                                                          14/04/2022 02:01  
ANO  MES SENADOR      TIPO_DESPESA                                       CNPJ_CPF           FORNECEDOR                    DOCUMENTO         DATA       DETALHAMENTO                                          COD_DOCUMENTO  
2022 1   ACIR GURGACZ Aluguel de imóveis para escritório político, co... 004.948.028-63     GILBERTO PISELO DO NASCIMENTO 001/22            03/01/2022 Despesa com pagamento de aluguel de imóvel para...          2173614  
                      Divulgação da atividade parlamentar                26.320.603/0001-64 INFORMANAHORA                 000000000000310/A 04/01/2022 Despesa com divulgação da atividade parlamentar...          2173615

* Os datasets estão com múltiplos indexadores, o que dificulta a limpeza e preparação dos dados.
* Irei criar novos dataframes com todos os indexadores desagrupados.

# Preparação e limpeza dos datasets.

## Desindexar colunas para fazer uma limpeza mais profunda nos dados.

In [12]:
despesa_ceaps_2019_desagrupado = despesa_ceaps_2019.reset_index()

despesa_ceaps_2020_desagrupado = despesa_ceaps_2020.reset_index()

despesa_ceaps_2021_desagrupado = despesa_ceaps_2021.reset_index()

despesa_ceaps_2022_desagrupado = despesa_ceaps_2022.reset_index()

### Renomear colunas

In [13]:
despesa_ceaps_2019_desagrupado.columns = list(despesa_ceaps_2019.index[0]) + list(despesa_ceaps_2019.columns)
despesa_ceaps_2019_desagrupado.columns = despesa_ceaps_2019_desagrupado.iloc[:1].values.tolist()[0]
despesa_ceaps_2019_desagrupado.drop([0], axis=0, inplace=True)

despesa_ceaps_2020_desagrupado.columns = list(despesa_ceaps_2020.index[0]) + list(despesa_ceaps_2020.columns)
despesa_ceaps_2020_desagrupado.columns = despesa_ceaps_2020_desagrupado.iloc[:1].values.tolist()[0]
despesa_ceaps_2020_desagrupado.drop([0], axis=0, inplace=True)

despesa_ceaps_2021_desagrupado.columns = list(despesa_ceaps_2021.index[0]) + list(despesa_ceaps_2021.columns)
despesa_ceaps_2021_desagrupado.columns = despesa_ceaps_2021_desagrupado.iloc[:1].values.tolist()[0]
despesa_ceaps_2021_desagrupado.drop([0], axis=0, inplace=True)

despesa_ceaps_2022_desagrupado.columns = list(despesa_ceaps_2022.index[0]) + list(despesa_ceaps_2022.columns)
despesa_ceaps_2022_desagrupado.columns = despesa_ceaps_2022_desagrupado.iloc[:1].values.tolist()[0]
despesa_ceaps_2022_desagrupado.drop([0], axis=0, inplace=True)

## Valores nulos

In [14]:
despesa_ceaps_2019_desagrupado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21634 entries, 1 to 21634
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ANO                21634 non-null  object
 1   MES                21634 non-null  object
 2   SENADOR            21634 non-null  object
 3   TIPO_DESPESA       21634 non-null  object
 4   CNPJ_CPF           21634 non-null  object
 5   FORNECEDOR         21634 non-null  object
 6   DOCUMENTO          20900 non-null  object
 7   DATA               21634 non-null  object
 8   DETALHAMENTO       13092 non-null  object
 9   VALOR_REEMBOLSADO  21634 non-null  object
 10  COD_DOCUMENTO      21634 non-null  object
dtypes: object(11)
memory usage: 2.0+ MB


* Os tipos de dados estão como object.
* Duas colunas apresentam valores nulos: 
  * DOCUMENTO e DETALHAMENTO.
* Hipóteses para valores nulos:
  * DOCUMENTO: 
    * erro no sistema.
    * Encobrir crimes.
  * DETALHAMENTO: 
    * Dependendo do tipo de despesa, não foi necessário apresentar o detalhamento.
    * O detalhamento está oculto por desvio de dinheiro público.

In [15]:
despesa_ceaps_2019_desagrupado[despesa_ceaps_2019_desagrupado['DETALHAMENTO']\
                               .isnull()][['TIPO_DESPESA']].value_counts()

TIPO_DESPESA                                                                                                                                                                                   
Locomoção, hospedagem, alimentação, combustíveis e lubrificantes                                                                                                                                   4391
Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.                                                                                                           2241
Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos.      789
Divulgação da atividade parlamentar                                                                                                                                                                 589
Contrata

* O tipo de despesa com mais dados faltantes da coluna DETALHAMENTO é:
  * Locomoção, hospedagem, alimentação, combustíveis e lubrificantes.
* O segundo com mais dados faltantes é:
  * Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.

In [16]:
despesa_ceaps_2019_desagrupado[despesa_ceaps_2019_desagrupado['DOCUMENTO']\
                               .isnull()][['TIPO_DESPESA']].value_counts()

TIPO_DESPESA                                                                                                                                                                                   
Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.                                                                                                           448
Locomoção, hospedagem, alimentação, combustíveis e lubrificantes                                                                                                                                   274
Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos.      12
dtype: int64

* O tipo de despesa com mais dados faltantes da coluna DOCUMENTO é:
Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.
* O segundo com mais dados faltantes é:
Locomoção, hospedagem, alimentação, combustíveis e lubrificantes.

* É necessário maior investigação para saber se os valores nulos são de fato acobertamento de um crime.

### A melhor estratégia é manter os valores nulos e criar novas colunas para indicar se há valores nulos nas colunas: DETALHAMENTO e DOCUMENTO.

In [17]:
def sem_documento(despesa_ceaps_2019_desagrupado):
  despesa_ceaps_2019_desagrupado['SEM_DOCUMENTO'] = np.nan
  for i in range(len(despesa_ceaps_2019_desagrupado['DOCUMENTO'])):
    if despesa_ceaps_2019_desagrupado['DOCUMENTO'].isnull().values.tolist()[i] == False:
      despesa_ceaps_2019_desagrupado['SEM_DOCUMENTO'][i] = 0
    else:
      despesa_ceaps_2019_desagrupado['SEM_DOCUMENTO'][i] = 1
  else:
    despesa_ceaps_2019_desagrupado['SEM_DOCUMENTO'] = despesa_ceaps_2019_desagrupado['SEM_DOCUMENTO'].fillna(0)     
    return despesa_ceaps_2019_desagrupado['SEM_DOCUMENTO']

def sem_detalhamento(despesa_ceaps_2019_desagrupado):
  despesa_ceaps_2019_desagrupado['SEM_DETALHAMENTO'] = np.nan
  for i in range(len(despesa_ceaps_2019_desagrupado['DETALHAMENTO'])):
    if despesa_ceaps_2019_desagrupado['DETALHAMENTO'].isnull().values.tolist()[i] == False:
      despesa_ceaps_2019_desagrupado['SEM_DETALHAMENTO'][i] = 0
    else:
      despesa_ceaps_2019_desagrupado['SEM_DETALHAMENTO'][i] = 1
  else:
    despesa_ceaps_2019_desagrupado['SEM_DETALHAMENTO'] = despesa_ceaps_2019_desagrupado['SEM_DETALHAMENTO'].fillna(0)     
    return despesa_ceaps_2019_desagrupado['SEM_DETALHAMENTO']

In [18]:
despesa_ceaps_2019_desagrupado['SEM_DOCUMENTO'] = sem_documento(despesa_ceaps_2019_desagrupado)
despesa_ceaps_2020_desagrupado['SEM_DOCUMENTO'] = sem_documento(despesa_ceaps_2020_desagrupado)
despesa_ceaps_2021_desagrupado['SEM_DOCUMENTO'] = sem_documento(despesa_ceaps_2021_desagrupado)
despesa_ceaps_2022_desagrupado['SEM_DOCUMENTO'] = sem_documento(despesa_ceaps_2022_desagrupado)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [19]:
despesa_ceaps_2019_desagrupado['SEM_DETALHAMENTO'] = sem_detalhamento(despesa_ceaps_2019_desagrupado)
despesa_ceaps_2020_desagrupado['SEM_DETALHAMENTO'] = sem_detalhamento(despesa_ceaps_2020_desagrupado)
despesa_ceaps_2021_desagrupado['SEM_DETALHAMENTO'] = sem_detalhamento(despesa_ceaps_2021_desagrupado)
despesa_ceaps_2022_desagrupado['SEM_DETALHAMENTO'] = sem_detalhamento(despesa_ceaps_2022_desagrupado)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Valores duplicados

In [20]:
print("despesa_2019:", 
      despesa_ceaps_2019_desagrupado.shape,
      "despesa_2019_sem_duplicados:", 
      despesa_ceaps_2019_desagrupado.drop_duplicates().shape)

print("despesa_2020:", 
      despesa_ceaps_2020_desagrupado.shape,
      "despesa_2020_sem_duplicados:", 
      despesa_ceaps_2020_desagrupado.drop_duplicates().shape)

print("despesa_2021:", 
      despesa_ceaps_2021_desagrupado.shape,
      "despesa_2021_sem_duplicados:", 
      despesa_ceaps_2021_desagrupado.drop_duplicates().shape)

print("despesa_2022:", 
      despesa_ceaps_2022_desagrupado.shape,
      "despesa_2022_sem_duplicados:", 
      despesa_ceaps_2022_desagrupado.drop_duplicates().shape)

despesa_2019: (21634, 13) despesa_2019_sem_duplicados: (21634, 13)
despesa_2020: (14090, 13) despesa_2020_sem_duplicados: (14090, 13)
despesa_2021: (16808, 13) despesa_2021_sem_duplicados: (16808, 13)
despesa_2022: (3749, 13) despesa_2022_sem_duplicados: (3749, 13)


* Todos os datasets não apresentam valores duplicados.

## Visão geral dos datasets.

In [21]:
despesa_ceaps_2019_desagrupado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21634 entries, 1 to 21634
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ANO                21634 non-null  object 
 1   MES                21634 non-null  object 
 2   SENADOR            21634 non-null  object 
 3   TIPO_DESPESA       21634 non-null  object 
 4   CNPJ_CPF           21634 non-null  object 
 5   FORNECEDOR         21634 non-null  object 
 6   DOCUMENTO          20900 non-null  object 
 7   DATA               21634 non-null  object 
 8   DETALHAMENTO       13092 non-null  object 
 9   VALOR_REEMBOLSADO  21634 non-null  object 
 10  COD_DOCUMENTO      21634 non-null  object 
 11  SEM_DOCUMENTO      21634 non-null  float64
 12  SEM_DETALHAMENTO   21634 non-null  float64
dtypes: float64(2), object(11)
memory usage: 2.8+ MB


In [22]:
despesa_ceaps_2020_desagrupado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14090 entries, 1 to 14090
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ANO                14090 non-null  object 
 1   MES                14090 non-null  object 
 2   SENADOR            14090 non-null  object 
 3   TIPO_DESPESA       14090 non-null  object 
 4   CNPJ_CPF           14090 non-null  object 
 5   FORNECEDOR         14090 non-null  object 
 6   DOCUMENTO          13470 non-null  object 
 7   DATA               14090 non-null  object 
 8   DETALHAMENTO       7228 non-null   object 
 9   VALOR_REEMBOLSADO  14090 non-null  object 
 10  COD_DOCUMENTO      14090 non-null  object 
 11  SEM_DOCUMENTO      14090 non-null  float64
 12  SEM_DETALHAMENTO   14090 non-null  float64
dtypes: float64(2), object(11)
memory usage: 2.0+ MB


In [23]:
despesa_ceaps_2021_desagrupado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16808 entries, 1 to 16808
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ANO                16808 non-null  object 
 1   MES                16808 non-null  object 
 2   SENADOR            16808 non-null  object 
 3   TIPO_DESPESA       16808 non-null  object 
 4   CNPJ_CPF           16808 non-null  object 
 5   FORNECEDOR         16808 non-null  object 
 6   DOCUMENTO          16133 non-null  object 
 7   DATA               16808 non-null  object 
 8   DETALHAMENTO       9193 non-null   object 
 9   VALOR_REEMBOLSADO  16808 non-null  object 
 10  COD_DOCUMENTO      16808 non-null  object 
 11  SEM_DOCUMENTO      16808 non-null  float64
 12  SEM_DETALHAMENTO   16808 non-null  float64
dtypes: float64(2), object(11)
memory usage: 2.3+ MB


In [24]:
despesa_ceaps_2022_desagrupado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3749 entries, 1 to 3749
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ANO                3749 non-null   object 
 1   MES                3749 non-null   object 
 2   SENADOR            3749 non-null   object 
 3   TIPO_DESPESA       3749 non-null   object 
 4   CNPJ_CPF           3749 non-null   object 
 5   FORNECEDOR         3749 non-null   object 
 6   DOCUMENTO          3583 non-null   object 
 7   DATA               3749 non-null   object 
 8   DETALHAMENTO       1907 non-null   object 
 9   VALOR_REEMBOLSADO  3749 non-null   object 
 10  COD_DOCUMENTO      3749 non-null   object 
 11  SEM_DOCUMENTO      3749 non-null   float64
 12  SEM_DETALHAMENTO   3749 non-null   float64
dtypes: float64(2), object(11)
memory usage: 539.1+ KB


In [25]:
despesa_ceaps_2019_desagrupado.head(3)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,SEM_DOCUMENTO,SEM_DETALHAMENTO
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,04/01/2019,Despesa com pagamento de energia elétrica do e...,"66,02",2116543,0.0,0.0
2,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,04/01/2019,Despesa com pagamento de energia elétrica do e...,"139,98",2116546,0.0,0.0
3,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,00119,07/01/2019,Despesa com pagamento de aluguel de imóvel par...,6000,2113817,0.0,0.0


In [26]:
despesa_ceaps_2019_desagrupado.tail(3)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,SEM_DOCUMENTO,SEM_DETALHAMENTO
21632,2019,1,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",07.575.651/0001-59,GOL,WNQI4A,29/01/2019,"Companhia Aérea: GOL, Localizador: WNQI4A. Pas...","1168,1",2114101,0.0,0.0
21633,2019,1,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",33.937.681/0001-78,LATAM,NCCPIM,23/01/2019,"Companhia Aérea: LATAM, Localizador: NCCPIM. P...","772,05",2113826,0.0,0.0
21634,2019,1,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",33.937.681/0001-78,LATAM,MEYLWB,26/01/2019,"Companhia Aérea: LATAM, Localizador: MEYLWB. P...","1755,17",2113830,0.0,0.0


In [27]:
despesa_ceaps_2020_desagrupado.head(3)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,SEM_DOCUMENTO,SEM_DETALHAMENTO
1,2020,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,14764742,06/01/2020,Despesa com energia elétrica para uso do escri...,"122,62",2141820,0.0,0.0
2,2020,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,14764741,06/01/2020,Despesa com energia elétrica para uso do escri...,"137,75",2141818,0.0,0.0
3,2020,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",062.135.728-64,FERNANDO WALDEIR PACINI e ANA LUCIA DA SILVA S...,001/2020,06/01/2020,Despesa com pagamento de aluguel de imóvel par...,1000,2139025,0.0,0.0


In [28]:
despesa_ceaps_2020_desagrupado.tail(3)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,SEM_DOCUMENTO,SEM_DETALHAMENTO
14088,2020,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,QIGVKT,14/12/2020,"Companhia Aérea: AZUL, Localizador: QIGVKT. Pa...","549,47",2153157,0.0,0.0
14089,2020,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,QDHKYF,14/12/2020,"Companhia Aérea: AZUL, Localizador: QDHKYF. Pa...","831,05",2153152,0.0,0.0
14090,2020,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,BQWCVK,18/12/2020,"Companhia Aérea: GOL, Localizador: BQWCVK. Pas...","2026,12",2153557,0.0,0.0


In [29]:
despesa_ceaps_2021_desagrupado.head(3)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,SEM_DOCUMENTO,SEM_DETALHAMENTO
1,2021,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,023.489.627,21/01/2021,Pagamento de energia elétrica para uso do escr...,"75,29",2158003,0.0,0.0
2,2021,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",062.135.728-64,FERNANDO WALDEIR PACINI e ANA LUCIA DA SILVA S...,01/2021,05/01/2021,Despesa com pagamento de aluguel de imóvel par...,1000,2157367,0.0,0.0
3,2021,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,001/21,06/01/2021,Despesa com aluguel de imóvel para uso do escr...,6000,2156383,0.0,0.0


In [30]:
despesa_ceaps_2021_desagrupado.tail(3)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,SEM_DOCUMENTO,SEM_DETALHAMENTO
16806,2021,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,SSKEOB,17/12/2021,"Companhia Aérea: GOL, Localizador: SSKEOB. Pas...","1486,66",2172717,0.0,0.0
16807,2021,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,ZICCEX,27/12/2021,"Companhia Aérea: LATAM, Localizador: ZICCEX. P...","2515,56",2173239,0.0,0.0
16808,2021,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,LM6FSK,29/12/2021,"Companhia Aérea: AZUL, Localizador: LM6FSK. Pa...","1510,75",2173238,0.0,0.0


* Pronto! Agora posso unir todos os datasets com em um só!

## Unir datasets.

In [31]:
despesa_ceaps_full = pd.concat([despesa_ceaps_2019_desagrupado, 
                    despesa_ceaps_2020_desagrupado, 
                    despesa_ceaps_2021_desagrupado, 
                    despesa_ceaps_2022_desagrupado], axis=0)

In [32]:
despesa_ceaps_full.head(3)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,SEM_DOCUMENTO,SEM_DETALHAMENTO
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,04/01/2019,Despesa com pagamento de energia elétrica do e...,"66,02",2116543,0.0,0.0
2,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,04/01/2019,Despesa com pagamento de energia elétrica do e...,"139,98",2116546,0.0,0.0
3,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,00119,07/01/2019,Despesa com pagamento de aluguel de imóvel par...,6000,2113817,0.0,0.0


## Transformar VALOR_REEMBOLSADO em float64.

In [33]:
despesa_ceaps_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56281 entries, 1 to 3749
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ANO                56281 non-null  object 
 1   MES                56281 non-null  object 
 2   SENADOR            56281 non-null  object 
 3   TIPO_DESPESA       56281 non-null  object 
 4   CNPJ_CPF           56281 non-null  object 
 5   FORNECEDOR         56281 non-null  object 
 6   DOCUMENTO          54086 non-null  object 
 7   DATA               56281 non-null  object 
 8   DETALHAMENTO       31420 non-null  object 
 9   VALOR_REEMBOLSADO  56281 non-null  object 
 10  COD_DOCUMENTO      56281 non-null  object 
 11  SEM_DOCUMENTO      56281 non-null  float64
 12  SEM_DETALHAMENTO   56281 non-null  float64
dtypes: float64(2), object(11)
memory usage: 6.0+ MB


In [34]:
despesa_ceaps_full['VALOR_REEMBOLSADO'] = despesa_ceaps_full['VALOR_REEMBOLSADO'].replace(',', '.', regex=True).astype('float')

despesa_ceaps_full[['ANO', 'MES']] = despesa_ceaps_full[['ANO', 'MES']].astype(int)

var_string = [col for col in despesa_ceaps_full if despesa_ceaps_full[col].dtype=='object']

despesa_ceaps_full[var_string] = despesa_ceaps_full[var_string].astype('string')

In [35]:
despesa_ceaps_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56281 entries, 1 to 3749
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ANO                56281 non-null  int64  
 1   MES                56281 non-null  int64  
 2   SENADOR            56281 non-null  string 
 3   TIPO_DESPESA       56281 non-null  string 
 4   CNPJ_CPF           56281 non-null  string 
 5   FORNECEDOR         56281 non-null  string 
 6   DOCUMENTO          54086 non-null  string 
 7   DATA               56281 non-null  string 
 8   DETALHAMENTO       31420 non-null  string 
 9   VALOR_REEMBOLSADO  56281 non-null  float64
 10  COD_DOCUMENTO      56281 non-null  string 
 11  SEM_DOCUMENTO      56281 non-null  float64
 12  SEM_DETALHAMENTO   56281 non-null  float64
dtypes: float64(3), int64(2), string(8)
memory usage: 6.0 MB


## Remover colunas que não serão analisadas.

In [36]:
features_drop = ['COD_DOCUMENTO', 'DOCUMENTO']
despesa_ceaps_full = despesa_ceaps_full.drop(features_drop, axis=1)

## Dataset Pronto para a EDA.

In [37]:
despesa_ceaps_full.head(3)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,SEM_DOCUMENTO,SEM_DETALHAMENTO
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,04/01/2019,Despesa com pagamento de energia elétrica do e...,66.02,0.0,0.0
2,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,04/01/2019,Despesa com pagamento de energia elétrica do e...,139.98,0.0,0.0
3,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,07/01/2019,Despesa com pagamento de aluguel de imóvel par...,6000.00,0.0,0.0


## Salvar Dataset

In [38]:
from google.colab import files
despesa_ceaps_full.to_csv('/gdrive/MyDrive/Colab Notebooks/#7DaysOfCode/despesa_ceaps_full.csv', sep=',')